In [2]:
%pip install pyarrow
%pip install pandas
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [19]:
import pandas as pd

# Чтение данных из файла Parquet
parquet_file_path = 'train_data/train_data/features.parquet'
video_data_from_parquet = pd.read_parquet(parquet_file_path)

# Создание нового DataFrame с первыми 1000 записями
sample_video_data = video_data_from_parquet.head(3000000)

# Сохранение нового DataFrame в файл
sample_file_path = 'train_data/train_data/sample_features_3000000.parquet'
sample_video_data.to_parquet(sample_file_path, index=False)

print(f"Создан новый файл: {sample_file_path}")

Создан новый файл: train_data/train_data/sample_features_3000000.parquet


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


def duration_to_seconds(duration):
    if isinstance(duration, int):
        return duration
    minutes, seconds = map(int, duration.split(':'))
    return minutes * 60 + seconds

# Чтение данных из файла Parquet
parquet_file_path = 'train_data/train_data/sample_features_100000.parquet'
video_data_from_parquet = pd.read_parquet(parquet_file_path)

# Добавление столбца 'IsClickbait' в DataFrame
video_data_from_parquet['IsClickbait'] = 0

# Переписать поле IsClickbait в соответствии с условиями
for index, row in video_data_from_parquet.iterrows():
    if row['v_cr_click_long_view_30_days'] < 0.2 and row['v_month_views'] > 100:
        video_data_from_parquet.at[index, 'IsClickbait'] = 1

# Вывести первые 10 строк для проверки изменений
# print(video_data_from_parquet[['video_id', 'v_cr_click_long_view_30_days', 'v_month_views', 'IsClickbait']].head(10))

# Создаем DataFrame на основе данных о видео
df = pd.DataFrame(video_data_from_parquet, columns=[
    'video_id', 'v_likes', 'v_dislikes', 'v_duration', 'v_cr_click_long_view_30_days', 'IsClickbait'])

# Заполняем пропущенные значения в столбце 'IsClickbait' и преобразовываем в целые числа
df['IsClickbait'] = df['IsClickbait'].replace([pd.NA, np.inf, -np.inf], np.nan).fillna(0).astype(int)

# Преобразование длительности видео в секунды
df['v_duration'] = df['v_duration'].apply(duration_to_seconds)

# Разделяем данные на признаки (X) и целевую переменную (y)
X = df[['v_likes', 'v_dislikes', 'v_duration', 'v_cr_click_long_view_30_days']]
y = df['IsClickbait']

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и обучаем модель случайного леса с взвешенными классами
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
clf.fit(X_train, y_train)

# Получаем предсказания на тестовом наборе
predictions = clf.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, predictions)
print(f"Точность модели: {accuracy}")

# Возвращаем видео, которые модель считает кликбейтами
clickbait_videos = df[df['IsClickbait'] == 1]

# Выводим первые 1000 video_id
print("Clickbait Video IDs:")
for video_id, v_cr_click_long_view_30_days in zip(
    clickbait_videos.head(100)['video_id'], clickbait_videos.head(100)['v_cr_click_long_view_30_days']):
    print(f"Video ID: {video_id} - {v_cr_click_long_view_30_days}")


Точность модели: 0.99745
Clickbait Video IDs:
Video ID: video_24631081 - 0.0
Video ID: video_15240794 - 0.1707317073170731
Video ID: video_30777930 - 0.1415094339622641
Video ID: video_27039098 - 0.0
Video ID: video_12561334 - 0.0075187969924812
Video ID: video_2673384 - 0.0
Video ID: video_10745836 - 0.1386138613861386
Video ID: video_31746503 - 0.0995260663507109
Video ID: video_3334699 - 0.1845238095238095
Video ID: video_29621775 - 0.1222826086956521
Video ID: video_10612924 - 0.1415094339622641
Video ID: video_29381612 - 0.0313725490196078
Video ID: video_22076231 - 0.0
Video ID: video_24021981 - 0.0366492146596858
Video ID: video_12859030 - 0.0035860655737704
Video ID: video_19346536 - 0.1666666666666666
Video ID: video_1543990 - 0.0
Video ID: video_18075793 - 0.0032608695652173
Video ID: video_10632791 - 0.0274223034734917
Video ID: video_14319756 - 0.1666666666666666
Video ID: video_11421819 - 0.0049019607843137
Video ID: video_25414623 - 0.1852941176470588
Video ID: video_5709